<a href="https://colab.research.google.com/github/Niharika08hub/Fake-News-Detection/blob/main/Niharika.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
os.makedirs("data/raw/images", exist_ok=True)

print("✅ Folder structure created")

✅ Folder structure created


In [ ]:
import os

zip_path = "Image_and_Text_Fake_News_Detection_Dataset.zip"
print("✅ Exists:", os.path.exists(zip_path))
print("📦 Size (bytes):", os.path.getsize(zip_path))


✅ Exists: True
📦 Size (bytes): 72351744


In [ ]:
from google.colab import files
import os
import zipfile
import mimetypes

# Step 1: Upload file
print("📂 Upload your dataset file (ZIP, CSV, etc.):")
uploaded = files.upload()

for filename in uploaded.keys():
    file_path = f"/content/{filename}"
    print(f"\n✅ Uploaded file: {filename}")

    # Step 2: Detect file type
    mime_type, _ = mimetypes.guess_type(file_path)
    print(f"Detected MIME type: {mime_type}")

    # Step 3: If ZIP, extract automatically
    if mime_type == 'application/zip':
        extract_path = "/content/data/raw"
        os.makedirs(extract_path, exist_ok=True)
        try:
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                bad_file = zip_ref.testzip()
                if bad_file:
                    print(f"❌ Corrupt file inside ZIP: {bad_file}")
                else:
                    zip_ref.extractall(extract_path)
                    print(f"✅ ZIP extracted successfully to {extract_path}!")
        except zipfile.BadZipFile:
            print("❌ The file is not a valid ZIP. Please re-download a proper .zip file.")
    else:
        print("ℹ️ Not a ZIP file. You can use it directly as-is at:", file_path)


📂 Upload your dataset file (ZIP, CSV, etc.):


Saving Image_and_Text_Fake_News_Detection_Dataset.zip to Image_and_Text_Fake_News_Detection_Dataset (1).zip

✅ Uploaded file: Image_and_Text_Fake_News_Detection_Dataset (1).zip
Detected MIME type: application/zip
✅ ZIP extracted successfully to /content/data/raw!


In [ ]:
import os

data_path = '/content/data/raw'
for root, dirs, files in os.walk(data_path):
    level = root.replace(data_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


raw/
    snopes_medical.zip
    fakeddit_subset.zip


In [ ]:
import zipfile
import os

# Paths
raw_path = '/content/data/raw'
extract_path = '/content/data/extracted'

os.makedirs(extract_path, exist_ok=True)

# Extract each zip
for zip_file in ['snopes_medical.zip', 'fakeddit_subset.zip']:
    zip_path = os.path.join(raw_path, zip_file)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"✅ Extracted {zip_file} to {extract_path}")


✅ Extracted snopes_medical.zip to /content/data/extracted
✅ Extracted fakeddit_subset.zip to /content/data/extracted


In [ ]:
import os

extracted_path = '/content/data/extracted'
for dataset in os.listdir(extracted_path):
    dataset_path = os.path.join(extracted_path, dataset)
    print(f"\nContents of {dataset}:")
    if os.path.isdir(dataset_path):
        for f in os.listdir(dataset_path):
            print(f" - {f}")
    else:
        print(f" - {dataset_path}")



Contents of fakeddit_subset:
 - training_data_fakeddit.jsonl
 - validation_data_fakeddit.jsonl
 - validation_image
 - image_folder

Contents of outputs:
 - images
 - data.csv


In [ ]:
import pandas as pd

# Load CSV
snopes_path = '/content/data/extracted/outputs/data.csv'
df = pd.read_csv(snopes_path)

# Quick look
print(df.head())

# Use correct columns
texts = df['Body'].astype(str)   # text content
labels = df['Rating']            # labels


   ID                                              Title  \
0   1  Can a Fetus Send Its Own Stem Cells to Repair ...   
1   2  Is a Testicular Blow Much More Painful Than Ch...   
2   3  Does Farting Negate the Use of Masks to Preven...   
3   4  Does Pfizer's COVID-19 Vaccine Contain Aborted...   
4   5  Is This 'Avoiding the Future Plague' Video a R...   

                                                Body       Rating        Date  \
0  A popular (but completely citation-free) scien...  Mostly True  07/12/2016   
1  A kick to the groin is, like childbirth, an un...        False  28/07/2016   
2    Despite overwhelming evidence that mask use ...        False  29/07/2020   
3  In October 2021, a number of social media user...        False  08/10/2021   
4    In September 2020, as the U.S. death toll fr...        False  05/10/2020   

                                               Image  
0  https://media.snopes.com/2016/12/embryonic_ste...  
1     https://media.snopes.com/2016/07/tes

In [ ]:
# Count samples per label
label_counts = labels.value_counts()
print(label_counts)


Rating
False                   217
True                     87
Mixture                  84
Unproven                 68
Mostly False             47
Mostly True              22
Miscaptioned             13
Correct Attribution      11
Labeled Satire            8
Outdated                  6
Legend                    4
Misattributed             2
Originated as Satire      1
Name: count, dtype: int64


In [ ]:
valid_labels = label_counts[label_counts > 1].index
df_filtered = df[df['Rating'].isin(valid_labels)]

texts = df_filtered['Body'].astype(str)
labels = df_filtered['Rating']


In [ ]:
# Keep labels with at least 2 samples
label_counts = df['Rating'].value_counts()
valid_labels = label_counts[label_counts >= 2].index
df_filtered = df[df['Rating'].isin(valid_labels)]

print("Filtered dataset size:", len(df_filtered))
print(df_filtered['Rating'].value_counts())


Filtered dataset size: 569
Rating
False                  217
True                    87
Mixture                 84
Unproven                68
Mostly False            47
Mostly True             22
Miscaptioned            13
Correct Attribution     11
Labeled Satire           8
Outdated                 6
Legend                   4
Misattributed            2
Name: count, dtype: int64


In [ ]:
texts = df_filtered['Body'].astype(str)
labels = df_filtered['Rating']


In [ ]:
from sklearn.model_selection import train_test_split

# If you still have very few samples per class, skip stratification
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


Training samples: 455
Testing samples: 114


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("TF-IDF features shape:", X_train_tfidf.shape)


TF-IDF features shape: (455, 5000)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Train MNB
mnb = MultinomialNB()
mnb.fit(X_train_tfidf, y_train)

# Predict
y_pred_mnb = mnb.predict(X_test_tfidf)

# Evaluate
print("✅ MNB Accuracy:", accuracy_score(y_test, y_pred_mnb))
print(classification_report(y_test, y_pred_mnb))


✅ MNB Accuracy: 0.38596491228070173
                     precision    recall  f1-score   support

Correct Attribution       0.00      0.00      0.00         2
              False       0.39      1.00      0.56        44
     Labeled Satire       0.00      0.00      0.00         2
             Legend       0.00      0.00      0.00         1
       Miscaptioned       0.00      0.00      0.00         3
            Mixture       0.00      0.00      0.00        17
       Mostly False       0.00      0.00      0.00         9
        Mostly True       0.00      0.00      0.00         4
           Outdated       0.00      0.00      0.00         1
               True       0.00      0.00      0.00        17
           Unproven       0.00      0.00      0.00        14

           accuracy                           0.39       114
          macro avg       0.04      0.09      0.05       114
       weighted avg       0.15      0.39      0.21       114



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.svm import LinearSVC

# Train SVM
svm = LinearSVC()
svm.fit(X_train_tfidf, y_train)

# Predict
y_pred_svm = svm.predict(X_test_tfidf)

# Evaluate
print("✅ SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


✅ SVM Accuracy: 0.37719298245614036
                     precision    recall  f1-score   support

Correct Attribution       0.00      0.00      0.00         2
              False       0.43      0.80      0.56        44
     Labeled Satire       0.00      0.00      0.00         2
             Legend       0.00      0.00      0.00         1
       Miscaptioned       0.00      0.00      0.00         3
            Mixture       0.33      0.24      0.28        17
       Mostly False       0.00      0.00      0.00         9
        Mostly True       0.00      0.00      0.00         4
           Outdated       0.00      0.00      0.00         1
               True       0.31      0.24      0.27        17
           Unproven       0.00      0.00      0.00        14

           accuracy                           0.38       114
          macro avg       0.10      0.12      0.10       114
       weighted avg       0.26      0.38      0.30       114



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

# Tokenization
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

# Tokenization
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)


In [ ]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    X_train_pad, y_train_enc,
    validation_split=0.1, epochs=5, batch_size=64
)

# Evaluate on test set
loss, acc = model.evaluate(X_test_pad, y_test_enc)
print("✅ LSTM Accuracy:", acc)


Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 629ms/step - accuracy: 0.2576 - loss: 2.4725 - val_accuracy: 0.4783 - val_loss: 2.3984
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 658ms/step - accuracy: 0.3567 - loss: 2.2974 - val_accuracy: 0.4783 - val_loss: 1.9005
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 571ms/step - accuracy: 0.3855 - loss: 1.9857 - val_accuracy: 0.4783 - val_loss: 1.8980
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 527ms/step - accuracy: 0.3737 - loss: 1.9246 - val_accuracy: 0.4783 - val_loss: 1.7759
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 574ms/step - accuracy: 0.3399 - loss: 1.8804 - val_accuracy: 0.4783 - val_loss: 1.7711
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.3783 - loss: 1.8778
✅ LSTM Accuracy: 0.38596490025520325


In [ ]:
# Predict probabilities on test set
y_pred_probs = model.predict(X_test_pad)

# Convert probabilities to class indices
import numpy as np
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Decode back to original string labels
y_pred_labels = le.inverse_transform(y_pred_classes)
y_test_labels = le.inverse_transform(y_test_enc)


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 345ms/step


In [ ]:
from sklearn.metrics import accuracy_score

acc_mnb = accuracy_score(y_test, y_pred_mnb)
acc_svm = accuracy_score(y_test, y_pred_svm)
acc_lstm = accuracy_score(y_test_labels, y_pred_labels)

print("MNB Accuracy:", acc_mnb)
print("SVM Accuracy:", acc_svm)
print("LSTM Accuracy:", acc_lstm)


MNB Accuracy: 0.38596491228070173
SVM Accuracy: 0.37719298245614036
LSTM Accuracy: 0.38596491228070173


In [ ]:
import pandas as pd

summary = pd.DataFrame({
    'Model': ['Multinomial Naive Bayes', 'SVM', 'LSTM'],
    'Accuracy': [acc_mnb, acc_svm, acc_lstm]
})

print(summary)


                     Model  Accuracy
0  Multinomial Naive Bayes  0.385965
1                      SVM  0.377193
2                     LSTM  0.385965


In [ ]:
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english', ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha=0.1)
mnb.fit(X_train_tfidf, y_train)


MultinomialNB(alpha=0.1)

In [ ]:
from sklearn.svm import LinearSVC
svm = LinearSVC(C=1.0, max_iter=5000)
svm.fit(X_train_tfidf, y_train)


LinearSVC(max_iter=5000)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=200),
    LSTM(64, dropout=0.3, recurrent_dropout=0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train_pad, y_train_enc, validation_split=0.1, epochs=10, batch_size=32)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 594ms/step - accuracy: 0.1810 - loss: 2.4775 - val_accuracy: 0.4783 - val_loss: 2.4167
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 240ms/step - accuracy: 0.3473 - loss: 2.3707 - val_accuracy: 0.4783 - val_loss: 1.8299
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 239ms/step - accuracy: 0.2909 - loss: 2.0682 - val_accuracy: 0.4783 - val_loss: 1.7933
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 341ms/step - accuracy: 0.3636 - loss: 1.9843 - val_accuracy: 0.4783 - val_loss: 1.7977
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 243ms/step - accuracy: 0.3860 - loss: 1.8703 - val_accuracy: 0.4783 - val_loss: 1.7839
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 244ms/step - accuracy: 0.3542 - loss: 1.8954 - val_accuracy: 0.4783 - val_loss: 1.7638
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 336ms/step - accuracy: 0.3797 - loss: 1.7406 - val_accuracy: 0.5000 - val_loss: 1.7844
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 243ms/step - accuracy: 0.4683 - loss: 1.6670 - val_accuracy: 0.5000 - val

In [ ]:
from sklearn.metrics import accuracy_score
import pandas as pd

acc_mnb = accuracy_score(y_test, y_pred_mnb) * 100
acc_svm = accuracy_score(y_test, y_pred_svm) * 100
acc_lstm = accuracy_score(y_test_labels, y_pred_labels) * 100

summary = pd.DataFrame({
    'Algorithm': ['Multinomial Naive Bayes', 'Support Vector Machine', 'Long Short-Term Memory'],
    'Accuracy (%)': [acc_mnb, acc_svm, acc_lstm]
})

print(summary)


                 Algorithm  Accuracy (%)
0  Multinomial Naive Bayes     38.596491
1   Support Vector Machine     37.719298
2   Long Short-Term Memory     38.596491


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


Training samples: 455
Testing samples: 114


In [ ]:
vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,2),
    stop_words='english'
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
mnb = MultinomialNB(alpha=0.1)
mnb.fit(X_train_tfidf, y_train)
y_pred_mnb = mnb.predict(X_test_tfidf)


In [ ]:
svm = LinearSVC(C=1.0, max_iter=5000)
svm.fit(X_train_tfidf, y_train)
y_pred_svm = svm.predict(X_test_tfidf)


In [ ]:
# Tokenize
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Encode labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)


In [ ]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, dropout=0.3, recurrent_dropout=0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_pad, y_train_enc, validation_split=0.1, epochs=10, batch_size=32)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 289ms/step - accuracy: 0.2718 - loss: 2.4572 - val_accuracy: 0.4783 - val_loss: 2.2965
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 240ms/step - accuracy: 0.3798 - loss: 2.1496 - val_accuracy: 0.4783 - val_loss: 1.7804
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 285ms/step - accuracy: 0.3543 - loss: 1.9225 - val_accuracy: 0.4783 - val_loss: 1.8281
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 241ms/step - accuracy: 0.3418 - loss: 1.9296 - val_accuracy: 0.4783 - val_loss: 1.7983
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 241ms/step - accuracy: 0.4190 - loss: 1.8141 - val_accuracy: 0.4783 - val_loss: 1.7788
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 334ms/step - accuracy: 0.3921 - loss: 1.7159 - val_accuracy: 0.4783 - val_loss: 1.7678
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 239ms/step - accuracy: 0.3765 - loss: 1.6996 - val_accuracy: 0.4783 - val_loss: 1.7700
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 243ms/step - accuracy: 0.4480 - loss: 1.5281 - val_accuracy: 0.3913 - val

In [ ]:
acc_mnb = accuracy_score(y_test, y_pred_mnb) * 100
acc_svm = accuracy_score(y_test, y_pred_svm) * 100
acc_lstm = accuracy_score(y_test_labels, y_pred_labels) * 100

summary = pd.DataFrame({
    'Algorithm': ['Multinomial Naive Bayes', 'Support Vector Machine', 'Long Short-Term Memory'],
    'Accuracy (%)': [acc_mnb, acc_svm, acc_lstm]
})

print(summary)


                 Algorithm  Accuracy (%)
0  Multinomial Naive Bayes     37.719298
1   Support Vector Machine     39.473684
2   Long Short-Term Memory     38.596491
